In [1]:
from hops import beam as hops_beam
# Start Beam jobservice
hops_beam.start(taskmanager_heap_size=8192)

{'jobserver_log': 'beamjobserver-p_1-runner-18120.log',
 'artifact_port': 22898,
 'expansion_port': 52754,
 'job_host': 'hopsworks0.logicalclocks.com',
 'job_port': 18120,
 'jobserver.pid': 8110}

In [2]:
import apache_beam as beam
from hops import beam as hops_beam
import os
from tfx.examples.chicago_taxi import preprocess
import shutil
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from google.protobuf import text_format
import tensorflow as tf
from typing import Text
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline

from tensorflow.python.lib.io import file_io

from tfx.examples.chicago_taxi.trainer import task
from tfx.examples.chicago_taxi.trainer import taxi
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

from hops import hdfs as hopsfs
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions, SetupOptions, HadoopFileSystemOptions, PortableOptions, WorkerOptions, DebugOptions



Instructions for updating:
from_feature_spec is a deprecated, use schema_utils.schema_from_feature_spec


In [7]:
_pipeline_name = 'chicago_taxi'
BASE_DIR = hopsfs.project_path(exclude_nn_addr=True)
_taxi_root = os.path.join(BASE_DIR, 'Resources')

hopsfs.mkdir('Resources/data/simple')

import requests

#Download dataset
url = 'https://github.com/tensorflow/tfx/raw/r0.14/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
r = requests.get(url, allow_redirects=True)
open('data.csv', 'wb').write(r.content)
# Copy data into resources
hopsfs.copy_to_hdfs('data.csv', 'Resources/data/simple', overwrite=True)

#Download utils
url = 'https://github.com/tensorflow/tfx/raw/r0.14/tfx/examples/chicago_taxi_pipeline/taxi_utils.py'
r = requests.get(url, allow_redirects=True)
open('taxi_utils.py', 'wb').write(r.content)
# Copy data into resources
hopsfs.copy_to_hdfs('taxi_utils.py', 'Resources', overwrite=True)

_data_root = os.path.join(_taxi_root, 'data', 'simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(_taxi_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model', _pipeline_name)

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['PYLIB'],'python3.6/site-packages/tfx')
_pipeline_root = os.path.join(_taxi_root, "pipelines", _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join("/tmp/metadata", _pipeline_name, 'metadata.db')

Started copying local path data.csv to hdfs path hdfs://10.0.2.15:8020/Projects/P_1/Resources/data/simple/data.csv

Finished copying

Started copying local path taxi_utils.py to hdfs path hdfs://10.0.2.15:8020/Projects/P_1/Resources/taxi_utils.py

Finished copying



In [8]:
pipeline_args = hops_beam.get_portable_runner_config()# + ['--requirements_file:None', '--requirements_cache:None', '--setup_file:None']
options=PipelineOptions(flags=pipeline_args)
print(pipeline_args)

['--runner=PortableRunner', '--hdfs_host=hopsworks0.logicalclocks.com', '--hdfs_port=50070', '--hdfs_user=P_1__meb10000', '--job_endpoint=hopsworks0.logicalclocks.com:18120', '--environment_type=PROCESS', '--environment_config={"command":"/srv/hops/flink/conf/sdk_worker.sh"}', '--sdk_worker_parallelism=1', '--experiments=worker_threads100', '--experiments=pre_optimize=all', '--execution_mode_for_batch=BATCH_FORCED']


In [9]:
def _create_pipeline(pipeline_name, pipeline_root, data_root,
                     module_file, serving_model_dir,
                     metadata_path):
  """Implements the chicago taxi pipeline with TFX."""
  examples = external_input(data_root)

  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = CsvExampleGen(input_base=examples)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(input_data=example_gen.outputs.examples)

  # Generates schema based on statistics files.
  infer_schema = SchemaGen(
      stats=statistics_gen.outputs.output, infer_feature_shape=False)

  # Performs anomaly detection based on statistics and data schema.
  validate_stats = ExampleValidator(
      stats=statistics_gen.outputs.output, schema=infer_schema.outputs.output)

  # Performs transformations and feature engineering in training and serving.
  transform = Transform(
      input_data=example_gen.outputs.examples,
      schema=infer_schema.outputs.output,
      module_file=module_file)

  # Uses user-provided Python function that implements a model using TF-Learn.
  trainer = Trainer(
      module_file=module_file,
      transformed_examples=transform.outputs.transformed_examples,
      schema=infer_schema.outputs.output,
      transform_output=transform.outputs.transform_output,
      train_args=trainer_pb2.TrainArgs(num_steps=10000),
      eval_args=trainer_pb2.EvalArgs(num_steps=5000))

  # Uses TFMA to compute a evaluation statistics over features of a model.
  model_analyzer = Evaluator(
      examples=example_gen.outputs.examples,
      model_exports=trainer.outputs.output,
      feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(specs=[
          evaluator_pb2.SingleSlicingSpec(
              column_for_slicing=['trip_start_hour'])
      ]))

  # Performs quality validation of a candidate model (compared to a baseline).
  model_validator = ModelValidator(
      examples=example_gen.outputs.examples, model=trainer.outputs.output)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = Pusher(
      model_export=trainer.outputs.output,
      model_blessing=model_validator.outputs.blessing,
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen, statistics_gen, infer_schema, validate_stats, transform,
          trainer, model_analyzer, model_validator, pusher
      ],
      enable_cache=False,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(
          metadata_path),
      additional_pipeline_args={
          'beam_pipeline_args': pipeline_args
      }
  )

In [10]:
# To run this pipeline from the python CLI:
#   $python taxi_pipeline_beam.py
if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.WARN)
  BeamDagRunner().run(
      _create_pipeline(
          pipeline_name=_pipeline_name,
          pipeline_root=_pipeline_root,
          data_root=_data_root,
          module_file=_module_file,
          serving_model_dir=_serving_model_dir,
          metadata_path=_metadata_path))
print('Done')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


...

value: "\n\014\n\nConst_10:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\014\n\nConst_10:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"



value: "\n\014\n\nConst_11:0\022-vocab_compute_and_apply_vocabulary_vocabulary"



value: "\n\014\n\nConst_11:0\022-vocab_compute_and_apply_vocabulary_vocabulary"



Done
